# Eat Safe, Love

## Notebook Set Up

In [157]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [158]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [159]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [160]:
# review the collections in our database
db.list_collection_names()

['establishments']

In [161]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [162]:
# Find the establishments with a hygiene score of 20
hygiene_score = 20
query = {"scores.Hygiene": hygiene_score}
parameters = {"BusinessName": 1, "_id": 0}
code_return = establishments.find(query, parameters)

##//test code to make sure the query worked properly\\##
# for document in code_return:
#     pprint(document)

# Use count_documents to display the number of documents in the result
result_count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of {hygiene_score}: {result_count}")
# Display the first document in the results using pprint
first_document = code_return.next()
pprint(first_document)

Number of establishments with a hygiene score of 20: 41
{'BusinessName': 'The Chase Rest Home'}


In [163]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(code_return)
# Display the number of rows in the DataFrame
num_rows = len(df)
print(f"Number of rows: {num_rows}")

# Display the first 10 rows of the DataFrame
df.head(10)

Number of rows: 40


,BusinessName
0,Brenalwood
1,Melrose Hotel
2,Seaford Pizza
3,Golden Palace
4,Ashby's Butchers
5,South Sea Express Cuisine
6,Golden Palace
7,The Tulip Tree
8,F & S
9,Longhouse


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [154]:
### another way to code this without using regex###
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
local_authority = "City of London Corporation"
min_rating_value = 4
query = {
    "LocalAuthorityName": local_authority,
    "RatingValue": {"$gte": min_rating_value}
}
parameters = {"BusinessName": 1, "_id": 0}
code_return = establishments.find(query, parameters)
# Use count_documents to display the number of documents in the result
result_count = establishments.count_documents(query)
print(f"Number of establishments in {local_authority} with a RatingValue greater than or equal to {min_rating_value}: {result_count}")
# Display the first document in the results using pprint
first_document = code_return.next()
pprint(first_document)


Number of establishments in City of London Corporation with a RatingValue greater than or equal to 4: 33
{'BusinessName': "Charlie's"}


In [155]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "RatingValue": {"$gte":4}, 
    "LocalAuthorityName": {"$regex": "City of London Corporation"}
}
parameters = {"BusinessName": 1, "_id": 0}
code_return = establishments.find(query, parameters)
# Use count_documents to display the number of documents in the result
result_count = establishments.count_documents(query)
print(result_count)
# Display the first document in the results using pprint
first_document = code_return.next()
pprint(first_document)


33
{'BusinessName': "Charlie's"}


In [156]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(code_return)
# Display the number of rows in the DataFrame
num_rows = (len(df))
print(num_rows)
# Display the first 10 rows of the DataFrame
df.head(10)

32


,BusinessName
0,Mv City Cruises Erasmus
1,Benfleet Motor Yacht Club
2,Coombs Catering t/a The Lock and Key
3,Tilbury Seafarers Centre
4,Mv Valulla
5,Tereza Joanne
6,Brick Lane Brews
7,The Nuance Group (UK) Limited
8,City Bar & Grill
9,WH Smith


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [182]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
# Penang Flavours: 'latitude': 51.490142, 'longitude': 0.08384
degree_search = 0.01
latitude = 0.08384
longitude = 51.490142

query = {"RatingValue": {"$eq":5}, 
         "geocode.latitude": {"$gt": (latitude - degree_search), "$lt": (latitude + degree_search)},
         "geocode.longitude": {"$gt": (longitude - degree_search), "$lt": (longitude + degree_search)}
         }
sort =  [("scores.Hygiene", 1)]
limit = 5

# Print the results
topFive = list(establishments.find(query).limit(limit))
pprint(topFive)


## I can't figure out why the above lines return an empty list... the code below appears to work better
degree_search = 0.01
latitude = 51.49
longitude = 0.08

# Construct the query
query = {
    "RatingValue": {"$eq": 5},
    "geocode": {
        "$near": [longitude, latitude],
        "$maxDistance": degree_search
    }
}
projection = {"BusinessName": 1, "scores.Hygiene": 1, "RatingValue": 1, "_id": 0}
sort =  [("scores.Hygiene", 1)]
# Print the results
topFive = list(establishments.find(query, projection).sort(sort).limit(5))
pprint(topFive)

[]
[{'BusinessName': 'BR6', 'RatingValue': 5, 'scores': {'Hygiene': 0}},
 {'BusinessName': 'Borna', 'RatingValue': 5, 'scores': {'Hygiene': 0}},
 {'BusinessName': 'Waterside Primary School',
  'RatingValue': 5,
  'scores': {'Hygiene': 0}},
 {'BusinessName': 'Catering & Hospitality Education Dep',
  'RatingValue': 5,
  'scores': {'Hygiene': 0}},
 {'BusinessName': 'Greenwich Islamic Centre',
  'RatingValue': 5,
  'scores': {'Hygiene': 0}}]


In [169]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(topFive)
df

,BusinessName,RatingValue,scores
0,BR6,5,{'Hygiene': 0}
1,Borna,5,{'Hygiene': 0}
2,Waterside Primary School,5,{'Hygiene': 0}
3,Catering & Hospitality Education Dep,5,{'Hygiene': 0}
4,Greenwich Islamic Centre,5,{'Hygiene': 0}


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [179]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {
        "$match": {"scores.Hygiene": 0}
    },
    {
        "$group": {
            "_id": "$LocalAuthorityName",
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {"count": -1}
    }
]
result = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
total_count = establishments.count_documents({"scores.Hygiene": 0})
print(total_count)
# Print the first 10 results
pprint(result[:10])


16827
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [180]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result)
# Display the number of rows in the DataFrame
num_rows = (len(df))
print(num_rows)
# Display the first 10 rows of the DataFrame
df.head(10)

55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
